<a href="https://colab.research.google.com/github/Bonnier98/DataJanitor-LLM/blob/main/consultor_de_vendasipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from typing import Dict, Any, Optional
import random

# --- BASES DE CONHECIMENTO ---
SCRIPTS = {
    'Interesse_Inicial': "Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?",
    'Resposta_Objetivo': "Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?",
    'Resposta_Historico_Treino': "E você já treinou antes ou está começando agora?",
    'Resposta_Conhece_Academia': "Você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    'Objecao_Preco_Digital': "Nossa, [Nome]! Temos planos a partir de R$ 49,90. Eu entendo que se sinta assim em relação ao plano, mas está caro comparado a que? Me diz para eu te ajudar a encontrar a melhor opção. 🧐",
    'Objecao_Sem_Tempo': "Super entendo! Por isso mesmo seria ótimo você vir 10 minutinhos só pra conhecer. Posso te mostrar os horários mais tranquilos. Prefere manhã ou tarde?",
    'Inativo_Identificado': "Oi [Nome]! Vi aqui que você já treinou com a gente 💛 Estamos com uma condição especial pra ex-alunos essa semana. Quer que eu te explique?",
    'Interesse_Promocao': "Que ótimo que você se interessou pela nossa promoção! Para te passar os valores e os detalhes exclusivos de [TITULO_PROMO_GENERICO], preciso saber se você já treinou conosco. Qual o seu nome?",
    'Transferencia_Consultor': "Entendi, [Nome]! Para te dar a melhor informação sobre a promoção/sua dúvida, vou pedir para uma de nossas consultoras te atender. Em 5 minutos, ela te explica tudo por aqui. Ok?",
    'Intencao_Nao_Identificada': "Me desculpe, ainda não entendi bem. Como consultor, preciso de mais detalhes para te dar a melhor solução."
}
CRM_INATIVOS = {'5511987654321': {'status': 'INATIVO', 'nome': 'Marta'}}
TITULO_PROMO_GENERICO = 'a Oferta Exclusiva deste Mês'
FRASES_HUMANIZACAO = ["Que massa saber disso!", "Você vai gostar muito do nosso ambiente.", "Fico feliz em te ajudar.", "Pode contar comigo."]


# --- CLASSE SalesAgentConsultivo (V21) ---
class SalesAgentConsultivo:

    def __init__(self):
        self.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'veio_da_promocao': False}
        self.nome_lead = ''
        print(f"Agente Consultivo ACADE FINAL (V21 - Correção Estrutural) inicializado.")

    def _format_script(self, script_key: str) -> str:
        # Scripts Especiais hardcoded no método
        if script_key == 'Encerramento_Suave_Special':
            script = "Que alegria saber que quer começar essa jornada com a gente, [Nome]! Vamos garantir que você alcance seus objetivos. Que tal deixar agendado sua visita para mês que vem? Prefere na primeira semana do mês ou segunda?"
        elif script_key == 'Objecao_Preco_Digital_Persistente':
            script = "Certo, [Nome]! Eu entendo a preocupação! Mas olha só: nossos planos começam a partir de R$ 49,90. Para que você tenha certeza de que é o investimento certo, posso te liberar 7 dias de acesso total ao ambiente e professores, sem compromisso. Topa vir conhecer e testar nosso trabalho?"
        else:
            script = SCRIPTS.get(script_key)

        nome_substituto = self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'
        script = script.replace('[Nome]', nome_substituto)
        script = script.replace('[TITULO_PROMO_GENERICO]', TITULO_PROMO_GENERICO)
        return script

    def _get_script(self, intent_name):
        if intent_name == 'Encerramento_Suave':
            return self._format_script('Encerramento_Suave_Special')
        if intent_name == 'Objecao_Preco_Digital_Persistente':
             return self._format_script('Objecao_Preco_Digital_Persistente')
        return self._format_script(intent_name)

    def add_human_touch(self, response: str) -> str:
        if random.random() < 0.25:
            toque = random.choice(FRASES_HUMANIZACAO)
            return f"{response}\n\n[Humanização: {toque}]"
        return response

    def check_lead_history(self, telefone: str) -> Optional[str]:
        if telefone in CRM_INATIVOS and CRM_INATIVOS[telefone]['status'] == 'INATIVO':
            self.nome_lead = CRM_INATIVOS[telefone]['nome']
            return 'Inativo_Identificado'
        return None

    def _simular_nlp(self, message: str) -> str:
        message_lower = message.lower()
        if any(keyword in message_lower for keyword in ['promoção', 'oferta', 'vi o link', 'desconto']): return 'Interesse_Promocao'
        if any(keyword in message_lower for keyword in ['tchau', 'agradeço', 'não vou seguir', 'mês que vem', 'parar por aqui', 'vou sair']): return 'Encerramento_Suave'
        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             if 'nome' in message_lower or len(message.split()) <= 4: return 'Resposta_Nome'
        if ('preço' in message_lower or 'valor' in message_lower or 'caro' in message_lower): return 'Objecao_Preco_Digital'
        if ('tempo' in message_lower or 'corrido' in message_lower or 'sem tempo' in message_lower): return 'Objecao_Sem_Tempo'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            termos_objetivo = ['músculos', 'musculos', 'musculo', 'emagrecer', 'saúde', 'condicionamento', 'foco']
            if any(termo in message_lower for termo in termos_objetivo): return 'Resposta_Objetivo'
        if self.contexto_lead['estado_atual'] == 'INICIO': return 'Interesse_Inicial'
        return 'Intencao_Nao_Identificada'


    def generate_response(self, client_message: str) -> str:

        # 1. TRATAMENTO INICIAL DE INATIVOS
        if self.contexto_lead['estado_atual'] == 'INICIO' and self.contexto_lead['telefone'] and not self.contexto_lead['inativo_tratado']:
            if self.check_lead_history(self.contexto_lead['telefone']) == 'Inativo_Identificado':
                self.contexto_lead['inativo_tratado'] = True
                self.contexto_lead['estado_atual'] = 'AGUARDANDO_OBJETIVO'
                return self._get_script('Inativo_Identificado')

        intent = self._simular_nlp(client_message)

        # 3. FLUXO DE PROMOÇÃO (Triagem e Transferência)
        if intent == 'Interesse_Promocao':
            if self.contexto_lead['estado_atual'] == 'INICIO':
                 self.contexto_lead['veio_da_promocao'] = True
                 self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
                 return self.add_human_touch(self._get_script('Interesse_Promocao'))
            elif self.nome_lead:
                self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
                return self.add_human_touch(self._get_script('Transferencia_Consultor'))

        # 4. FLUXO DE ENCERRAMENTO
        elif intent == 'Encerramento_Suave':
            self.contexto_lead['estado_atual'] = 'FIM'
            return self._get_script('Encerramento_Suave')

        # 5. FLUXO DE CAPTURA DE NOME
        elif intent == 'Resposta_Nome' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
            self.contexto_lead['objecao_preco_contagem'] = 0
            nome_candidato = client_message.split()[-1].replace('.', '').replace(',', '')
            if len(nome_candidato) > 2 and nome_candidato.lower() not in ['sim', 'não', 'eu', 'ok', 'aqui']: self.nome_lead = nome_candidato

            if self.contexto_lead['veio_da_promocao']:
                self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
                return self.add_human_touch(self._get_script('Transferencia_Consultor'))

            self.contexto_lead['estado_atual'] = 'AGUARDANDO_OBJETIVO'
            return self._get_script('Resposta_Objetivo')

        # 6. FLUXO DE OBJEÇÕES (Persistência)
        elif intent in ['Objecao_Preco_Digital', 'Objecao_Sem_Tempo']:
            if intent == 'Objecao_Preco_Digital':
                self.contexto_lead['objecao_preco_contagem'] += 1
                if self.contexto_lead['objecao_preco_contagem'] > 1:
                    return self.add_human_touch(self._get_script('Objecao_Preco_Digital_Persistente'))

            if self.contexto_lead['estado_atual'] == 'INICIO':
                self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self.add_human_touch(self._get_script(intent))

        # 7. FLUXO DE INTERESSE INICIAL / RESPOSTA OBJETIVO (Avanço de Fluxo)
        elif intent == 'Interesse_Inicial':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self.add_human_touch(self._get_script('Interesse_Inicial'))

        elif intent == 'Resposta_Objetivo' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_HISTORICO'
            return self.add_human_touch(self._get_script('Resposta_Historico_Treino'))

        elif intent == 'Resposta_Historico_Treino' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_VISITA'
            return self.add_human_touch(self._get_script('Resposta_Conhece_Academia'))

        # 8. FALLBACK UNIVERSAL -> TRANSFIRA PARA HUMANO
        elif intent == 'Intencao_Nao_Identificada':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self.add_human_touch(self._get_script('Transferencia_Consultor'))

        return self._get_script('Intencao_Nao_Identificada')


# --- FUNÇÕES DE TESTE ---

def testar_fluxo(agente_ia, nome_do_teste: str, mensagens: Dict[str, str]):
    """Função para simular um fluxo de conversa completo."""
    print("=====================================================")
    print(f"✅ INICIANDO TESTE: {nome_do_teste}")
    print("=====================================================")

    # Reseta o contexto
    agente_ia.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'veio_da_promocao': False}
    agente_ia.nome_lead = ''

    for idx, (cliente_msg, intent_esperada) in enumerate(mensagens.items()):
        agente_response = agente_ia.generate_response(cliente_msg)

        print(f"\n[{idx+1}] CLIENTE: {cliente_msg}")
        print

In [ ]:
# Reutilizando a classe e variáveis globais do Bloco 1

# --- FUNÇÕES DE TESTE ---
def testar_fluxo(agente_ia, nome_do_teste: str, mensagens: Dict[str, str]):
    """Função para simular um fluxo de conversa completo."""
    print("=====================================================")
    print(f"✅ INICIANDO TESTE: {nome_do_teste}")
    print("=====================================================")

    # Reseta o contexto
    agente_ia.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'veio_da_promocao': False}
    agente_ia.nome_lead = ''

    for idx, (cliente_msg, intent_esperada) in enumerate(mensagens.items()):
        agente_response = agente_ia.generate_response(cliente_msg)

        print(f"\n[{idx+1}] CLIENTE: {cliente_msg}")
        print(f"    AGENTE ({agente_ia.contexto_lead['estado_atual']}): {agente_response.splitlines()[0]}...")

        if intent_esperada in agente_response:
             print(f"    [SUCESSO] Contém: '{intent_esperada.splitlines()[0]}'")
        else:
             print(f"    [FALHA] Resposta inesperada. Esperado conter: '{intent_esperada.splitlines()[0]}'")


# --- EXECUÇÃO DOS TESTES FINAIS ---

print("\n--- TESTE DE COBERTURA FINAL (V19) ---")
agente_final = SalesAgentConsultivo()

# 1. FLUXO DE VENDA CONSULTIVA (Objeção e Persistência)
fluxo_venda = {
    "Olá, queria saber sobre os planos.": "Boa tarde!",
    "Meu nome é Rafaela.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Quero ganhar massa muscular.": "E você já treinou antes ou está começando agora?",
    "Achei o preço caro.": "Nossa, Rafaela! Temos planos a partir de R$ 49,90.",
    "Ainda está pesado para mim.": "Certo, Rafaela! Eu entendo a preocupação! Mas olha só: nossos planos começam",
    "Vou pensar e te aviso.": "Que alegria saber que quer começar essa jornada com a gente, Rafaela!",
}
testar_fluxo(agente_final, "Venda Consultiva (Objeção com Persistência)", fluxo_venda)

# 2. FLUXO DE PROMOÇÃO E TRANSFERÊNCIA HUMANA
fluxo_promocao = {
    "Vi a promoção do mês, como funciona?": "Que ótimo que você se interessou pela nossa promoção!",
    "Meu nome é Pedro.": "Entendi, Pedro! Para te dar a melhor informação sobre a promoção/sua dúvida, vou pedir para uma de nossas consultoras te atender.",
}
testar_fluxo(agente_final, "Promoção e Transferência Humana", fluxo_promocao)


--- TESTE DE COBERTURA FINAL (V19) ---
Agente Consultivo ACADE FINAL (V21 - Correção Estrutural) inicializado.
✅ INICIANDO TESTE: Venda Consultiva (Objeção com Persistência)

[1] CLIENTE: Olá, queria saber sobre os planos.
    AGENTE (AGUARDANDO_NOME): Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?...
    [SUCESSO] Contém: 'Boa tarde!'

[2] CLIENTE: Meu nome é Rafaela.
    AGENTE (AGUARDANDO_OBJETIVO): Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?...
    [SUCESSO] Contém: 'Pra eu te orientar direitinho, qual é o seu objetivo hoje?'

[3] CLIENTE: Quero ganhar massa muscular.
    AGENTE (TRANSFERIDO): Entendi, Rafaela! Para te dar a melhor informação sobre a promoção/sua dúvida, vou pedir para uma de nossas consultoras te atender. Em 5 minutos, ela te explica tudo por aqui. Ok?...
    [FALHA] Resposta inesperada. Esperado conter: 'E vo

In [ ]:
import pandas as pd
from typing import Dict, Any, Optional
import random

# --- BASES DE CONHECIMENTO ---
SCRIPTS = {
    'Interesse_Inicial': "Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?",
    'Resposta_Objetivo': "Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?",
    'Resposta_Historico_Treino': "E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')",
    'Resposta_Conhece_Academia': "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    'Objecao_Preco_Digital': "Nossa, [Nome]! Temos planos a partir de R$ 49,90. Eu entendo que se sinta assim em relação ao plano, mas está caro comparado a que? Me diz para eu te ajudar a encontrar a melhor opção. 🧐",
    'Objecao_Sem_Tempo': "Super entendo! Por isso mesmo seria ótimo você vir 10 minutinhos só pra conhecer. Posso te mostrar os horários mais tranquilos. Prefere manhã ou tarde?",
    'Inativo_Identificado': "Oi [Nome]! Vi aqui que você já treinou com a gente 💛 Estamos com uma condição especial pra ex-alunos essa semana. Quer que eu te explique?",
    'Transferencia_Consultor': "Certo, [Nome]. Para garantir que todas as suas dúvidas sejam sanadas, vou pedir para uma de nossas consultoras te atender em 5 minutos. Ok?",
    'Intencao_Nao_Identificada': "Me desculpe, ainda não entendi bem. Como consultor, preciso de mais detalhes para te dar a melhor solução."
}
CRM_INATIVOS = {'5511987654321': {'status': 'INATIVO', 'nome': 'Marta'}}
FRASES_HUMANIZACAO = ["Que massa saber disso!", "Você vai gostar muito do nosso ambiente.", "Fico feliz em te ajudar.", "Pode contar comigo."]


# --- CLASSE SalesAgentConsultivo (V23) ---
class SalesAgentConsultivo:

    def __init__(self):
        # Contexto atualizado sem variáveis de promoção
        self.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
        self.nome_lead = ''
        print(f"Agente Consultivo ACADE FINAL (V23 - Lógica Corrigida) inicializado.")

    def _format_script(self, script_key: str) -> str:

        # Scripts Especiais hardcoded
        if script_key == 'Offer_Novo_Aluno':
             script = "Que ótimo! Para que você veja a qualidade das nossas aulas e o ambiente, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"
        elif script_key == 'Offer_Ex_Aluno':
             script = "Que bom que você pensa em voltar, [Nome]! Desde que você saiu, reformamos tudo e temos equipamentos novos! Para você conferir, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"
        elif script_key == 'Reagendamento_Objeção_Pessoal':
            objetivo = self.contexto_lead.get('objetivo_pessoal_completo', 'ser mais saudável').lower().split(' ')[-1] # Tenta pegar a última palavra
            script = f"Entendi sua preocupação, [Nome]. Mas você mencionou que o objetivo é **{self.contexto_lead.get('objetivo_pessoal_completo', 'ser mais saudável')}**. Se você adiar, vai ficar mais difícil atingir essa meta. Vamos agendar sua visita para hoje ou amanhã para garantir que você não perca o foco?"
        elif script_key == 'Encerramento_Suave':
            # Mantém script suave, que pode ser a última tentativa de agendamento.
            script = "Que alegria saber que quer começar essa jornada com a gente, [Nome]! Vamos garantir que você alcance seus objetivos. Que tal deixar agendado sua visita para mês que vem? Prefere na primeira semana do mês ou segunda?"
        else:
            script = SCRIPTS.get(script_key)

        nome_substituto = self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'
        script = script.replace('[Nome]', nome_substituto)
        return script

    def _get_script(self, intent_name):
        if intent_name == 'Reagendamento_Objeção_Pessoal':
            return self._format_script(intent_name)
        if intent_name == 'Offer_Ex_Aluno':
             return self._format_script(intent_name)
        if intent_name == 'Offer_Novo_Aluno':
             return self._format_script(intent_name)
        if intent_name == 'Encerramento_Suave':
             return self._format_script(intent_name)
        return self._format_script(intent_name)

    def add_human_touch(self, response: str) -> str:
        if random.random() < 0.25:
            toque = random.choice(FRASES_HUMANIZACAO)
            return f"{response}\n\n[Humanização: {toque}]"
        return response

    def check_lead_history(self, telefone: str) -> Optional[str]:
        if telefone in CRM_INATIVOS and CRM_INATIVOS[telefone]['status'] == 'INATIVO':
            self.nome_lead = CRM_INATIVOS[telefone]['nome']
            return 'Inativo_Identificado'
        return None

    def _simular_nlp(self, message: str) -> str:
        message_lower = message.lower()

        # 1. Objeções e Encerramento (Alta Prioridade)
        if any(keyword in message_lower for keyword in ['tchau', 'agradeço', 'não vou seguir', 'parar por aqui', 'vou sair', 'não quero de forma alguma']): return 'Recusa_Total'
        if ('preço' in message_lower or 'valor' in message_lower or 'caro' in message_lower): return 'Objecao_Preco_Digital'
        if ('tempo' in message_lower or 'corrido' in message_lower or 'sem tempo' in message_lower): return 'Objecao_Sem_Tempo'

        # 2. Respostas de Fluxo
        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             if 'nome' in message_lower or len(message.split()) <= 4: return 'Resposta_Nome'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            termos_objetivo = ['músculos', 'musculos', 'musculo', 'emagrecer', 'saúde', 'condicionamento', 'foco', 'massa']
            if any(termo in message_lower for termo in termos_objetivo):
                self.contexto_lead['objetivo_pessoal_completo'] = message # Captura a frase completa
                return 'Resposta_Objetivo'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            if any(keyword in message_lower for keyword in ['ex-aluno', 'já fui', 'treinei antes', 'antigo aluno']):
                self.contexto_lead['historico_treino'] = 'EX_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            if any(keyword in message_lower for keyword in ['nunca', 'primeira vez', 'começando agora']):
                self.contexto_lead['historico_treino'] = 'NOVO_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            # Qualquer outra resposta no estado AGUARDANDO_HISTORICO avança o fluxo
            return 'Resposta_Historico_Treino_Geral'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
             # Qualquer resposta no estado AGUARDANDO_VISITA avança para o Agendamento
             return 'Resposta_Conhece_Academia'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_AGENDAMENTO':
             # Resposta de Agendamento (sim ou tempo)
             if any(keyword in message_lower for keyword in ['hoje', 'amanhã', 'tarde', 'manhã', 'agora', 'sim', 'quero', 'pode ser']):
                 return 'Agendamento_Sucesso'

        # 3. Tratamento de Fluxo Padrão
        if self.contexto_lead['estado_atual'] == 'INICIO': return 'Interesse_Inicial'

        return 'Intencao_Nao_Identificada'


    def generate_response(self, client_message: str) -> str:

        # 0. Checagem de estado final/transferido (para evitar looping)
        if self.contexto_lead['estado_atual'] in ['TRANSFERIDO', 'FIM']:
             return self._get_script('Transferencia_Consultor')

        # 1. TRATAMENTO INICIAL DE INATIVOS (Checa no INICIO)
        if self.contexto_lead['estado_atual'] == 'INICIO' and self.contexto_lead['telefone'] and not self.contexto_lead['inativo_tratado']:
            if self.check_lead_history(self.contexto_lead['telefone']) == 'Inativo_Identificado':
                self.contexto_lead['inativo_tratado'] = True
                self.contexto_lead['estado_atual'] = 'AGUARDANDO_OBJETIVO'
                return self._get_script('Inativo_Identificado')

        intent = self._simular_nlp(client_message)

        # 2. FLUXO DE ENCERRAMENTO (Recusa total ou Encerramento Suave)
        if intent == 'Recusa_Total':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO' # Marca para transferência
            return self.add_human_touch(self._get_script('Transferencia_Consultor'))

        elif intent == 'Encerramento_Suave':
            self.contexto_lead['estado_atual'] = 'FIM'
            return self._get_script('Encerramento_Suave') # Script suave de reagendamento

        # 3. FLUXO DE CAPTURA DE NOME
        elif intent == 'Resposta_Nome' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
            self.contexto_lead['objecao_preco_contagem'] = 0
            nome_candidato = client_message.split()[-1].replace('.', '').replace(',', '')
            if len(nome_candidato) > 2 and nome_candidato.lower() not in ['sim', 'não', 'eu', 'ok', 'aqui']: self.nome_lead = nome_candidato

            self.contexto_lead['estado_atual'] = 'AGUARDANDO_OBJETIVO'
            return self._get_script('Resposta_Objetivo')

        # 4. FLUXO DE OBJEÇÕES (Persistência com Personalização)
        elif intent in ['Objecao_Preco_Digital', 'Objecao_Sem_Tempo']:
            if intent == 'Objecao_Preco_Digital':
                self.contexto_lead['objecao_preco_contagem'] += 1

                # Persistência Nível 1 (Personalização)
                if self.contexto_lead['objecao_preco_contagem'] == 1:
                     return self.add_human_touch(self._get_script('Reagendamento_Objeção_Pessoal'))

                # Persistência Nível 2 (7 Dias Grátis)
                if self.contexto_lead['objecao_preco_contagem'] == 2:
                    return self.add_human_touch(self._format_script('Objecao_Preco_Digital_Persistente'))

            if self.contexto_lead['estado_atual'] == 'INICIO':
                self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'

            return self.add_human_touch(self._get_script(intent))

        # 5. FLUXO DE INTERESSE INICIAL
        elif intent == 'Interesse_Inicial':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self.add_human_touch(self._get_script('Interesse_Inicial'))

        # 6. FLUXO DE RESPOSTA OBJETIVO -> AGUARDANDO HISTORICO
        elif intent == 'Resposta_Objetivo' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_HISTORICO'
            return self.add_human_touch(self._get_script('Resposta_Historico_Treino'))

        # 7. FLUXO DE RESPOSTA HISTÓRICO -> CONHECE ACADEMIA
        elif intent in ['Resposta_Historico_Treino_Especifica', 'Resposta_Historico_Treino_Geral'] and self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_VISITA'
            return self.add_human_touch(self._get_script('Resposta_Conhece_Academia'))

        # 8. FLUXO DE RESPOSTA CONHECE ACADEMIA -> OFERTA DE 7 DIAS + AGENDAMENTO
        elif intent == 'Resposta_Conhece_Academia' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_AGENDAMENTO'

            if self.contexto_lead.get('historico_treino') == 'EX_ALUNO':
                 return self.add_human_touch(self._get_script('Offer_Ex_Aluno'))
            else:
                 return self.add_human_touch(self._get_script('Offer_Novo_Aluno'))

        # 9. FLUXO DE AGENDAMENTO BEM SUCEDIDO
        elif intent == 'Agendamento_Sucesso' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_AGENDAMENTO':
            self.contexto_lead['estado_atual'] = 'FIM'
            return self.add_human_touch(f"Perfeito, {self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'}! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳")

        # 10. FALLBACK UNIVERSAL -> TRANSFIRA PARA HUMANO
        elif intent == 'Intencao_Nao_Identificada':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self.add_human_touch(self._get_script('Transferencia_Consultor'))

        return self._get_script('Intencao_Nao_Identificada')


# --- FUNÇÕES DE TESTE ---

def testar_fluxo(agente_ia, nome_do_teste: str, mensagens: Dict[str, str]):
    """Função para simular um fluxo de conversa completo."""
    print("=====================================================")
    print(f"✅ INICIANDO TESTE: {nome_do_teste}")
    print("=====================================================")

    # Reseta o contexto
    agente_ia.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
    agente_ia.nome_lead = ''

    for idx, (cliente_msg, intent_esperada) in enumerate(mensagens.items()):
        agente_response = agente_ia.generate_response(cliente_msg)

        print(f"\n[{idx+1}] CLIENTE: {cliente_msg}")
        print(f"    AGENTE ({agente_ia.contexto_lead['estado_atual']}): {agente_response.splitlines()[0]}...")

        if intent_esperada in agente_response:
             print(f"    [SUCESSO] Contém: '{intent_esperada.splitlines()[0]}'")
        else:
             print(f"    [FALHA] Resposta inesperada. Esperado conter: '{intent_esperada.splitlines()[0]}'")


# --- EXECUÇÃO DOS TESTES FINAIS ---

print("\n--- TESTE DE COBERTURA FINAL (V23) ---")
agente_final = SalesAgentConsultivo()

# 1. FLUXO DE VENDA CONSULTIVA (Personalização e Objeção)
fluxo_venda_personalizada = {
    "Olá, queria saber sobre os planos.": "Boa tarde!",
    "Meu nome é Rafaela.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Quero emagrecer porque quero usar biquine final de ano.": "E você já treinou antes ou está começando agora?",
    "Já fui aluno, mas faz 2 anos.": "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    "Eu conheço sim.": "Que bom que você pensa em voltar, Rafaela! Desde que você saiu, reformamos tudo e temos equipamentos novos!", # Offer_Ex_Aluno
    "Achei o preço caro.": "Entendi sua preocupação, Rafaela. Mas você mencionou que o objetivo é **Quero emagrecer porque quero usar biquine final de ano**.", # Persuasão com Objeção Pessoal
    "Ainda está pesado para mim.": "Certo, Rafaela! Eu entendo a preocupação! Mas olha só: nossos planos começam", # Objeção Persistente (7 dias grátis)
    "Claro que sim, agendamos para amanhã de manhã.": "Perfeito, Rafaela! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳",
}
testar_fluxo(agente_final, "Venda Consultiva (Persuasão e Ex-Aluno)", fluxo_venda_personalizada)

# 2. FLUXO DE NOVO ALUNO COM RECUSA TOTAL
fluxo_novo_aluno = {
    "Tenho interesse na academia.": "Boa tarde!",
    "Meu nome é Pedro.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Quero ganhar massa e ficar forte.": "E você já treinou antes ou está começando agora?",
    "Nunca treinei.": "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    "Não conheço, primeira vez.": "Que ótimo! Para que você veja a qualidade das nossas aulas e o ambiente, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?", # Offer_Novo_Aluno
    "Não quero de forma alguma, vou ver outras academias.": "Certo, Pedro. Para garantir que todas as suas dúvidas sejam sanadas, vou pedir para uma de nossas consultoras te atender em 5 minutos. Ok?",
}
testar_fluxo(agente_final, "Novo Aluno e Recusa Total (Transferência)", fluxo_novo_aluno)



--- TESTE DE COBERTURA FINAL (V23) ---
Agente Consultivo ACADE FINAL (V23 - Lógica Corrigida) inicializado.
✅ INICIANDO TESTE: Venda Consultiva (Persuasão e Ex-Aluno)

[1] CLIENTE: Olá, queria saber sobre os planos.
    AGENTE (AGUARDANDO_NOME): Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?...
    [SUCESSO] Contém: 'Boa tarde!'

[2] CLIENTE: Meu nome é Rafaela.
    AGENTE (AGUARDANDO_OBJETIVO): Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?...
    [SUCESSO] Contém: 'Pra eu te orientar direitinho, qual é o seu objetivo hoje?'

[3] CLIENTE: Quero emagrecer porque quero usar biquine final de ano.
    AGENTE (AGUARDANDO_HISTORICO): E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')...
    [SUCESSO] Contém: 'E você já treinou antes ou está começando agora?'

[4] CLIENTE: Já fui aluno, mas faz 2 

In [ ]:
import pandas as pd
from typing import Dict, Any, Optional
import random

# --- BASES DE CONHECIMENTO ---
SCRIPTS = {
    'Interesse_Inicial': "Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?",
    'Resposta_Objetivo': "Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?",
    'Resposta_Historico_Treino': "E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')",
    'Resposta_Conhece_Academia': "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    'Objecao_Preco_Digital': "Nossa, [Nome]! Temos planos a partir de R$ 49,90. Eu entendo que se sinta assim em relação ao plano, mas está caro comparado a que? Me diz para eu te ajudar a encontrar a melhor opção. 🧐",
    'Objecao_Sem_Tempo': "Super entendo! Por isso mesmo seria ótimo você vir 10 minutinhos só pra conhecer. Posso te mostrar os horários mais tranquilos. Prefere manhã ou tarde?",
    'Inativo_Identificado': "Oi [Nome]! Vi aqui que você já treinou com a gente 💛 Estamos com uma condição especial pra ex-alunos essa semana. Quer que eu te explique?",
    'Transferencia_Consultor': "Certo, [Nome]. Para garantir que todas as suas dúvidas sejam sanadas, vou pedir para uma de nossas consultoras te atender em 5 minutos. Ok?",
    'Intencao_Nao_Identificada': "Me desculpe, ainda não entendi bem. Como consultor, preciso de mais detalhes para te dar a melhor solução."
}
CRM_INATIVOS = {'5511987654321': {'status': 'INATIVO', 'nome': 'Marta'}}
FRASES_HUMANIZACAO = ["Que massa saber disso!", "Você vai gostar muito do nosso ambiente.", "Fico feliz em te ajudar.", "Pode contar comigo."]


# --- CLASSE SalesAgentConsultivo (V31) ---
class SalesAgentConsultivo:

    def __init__(self):
        self.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
        self.nome_lead = ''
        print(f"Agente Consultivo ACADE FINAL (V31 - Handler Objeção) inicializado.")

    def _format_script(self, script_key: str) -> str:

        if script_key == 'Offer_Novo_Aluno':
             script = "Que ótimo! Para que você veja a qualidade das nossas aulas e o ambiente, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"
        elif script_key == 'Offer_Ex_Aluno':
             script = "Que bom que você pensa em voltar, [Nome]! Desde que você saiu, reformamos tudo e temos equipamentos novos! Para você conferir, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"
        elif script_key == 'Reagendamento_Objeção_Pessoal':
            objetivo_limpo = self.contexto_lead.get('objetivo_pessoal_completo', 'ser mais saudável').strip().rstrip('.')
            script = f"Entendi sua preocupação, [Nome]. Mas você mencionou que o objetivo é **{objetivo_limpo}**. Se você adiar, vai ficar mais difícil atingir essa meta. Vamos agendar sua visita para hoje ou amanhã para garantir que você não perca o foco?"
        elif script_key == 'Encerramento_Suave':
            script = "Que alegria saber que quer começar essa jornada com a gente, [Nome]! Vamos garantir que você alcance seus objetivos. Que tal deixar agendado sua visita para mês que vem? Prefere na primeira semana do mês ou segunda?"
        elif script_key == 'Objecao_Preco_Digital_Persistente':
            script = "Certo, [Nome]! Eu entendo a preocupação! Mas olha só: nossos planos começam a partir de R$ 49,90. Para que você tenha certeza de que é o investimento certo, posso te liberar 7 dias de acesso total ao ambiente e professores, sem compromisso. Topa vir conhecer e testar nosso trabalho?"
        else:
            script = SCRIPTS.get(script_key)

        nome_substituto = self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'
        script = script.replace('[Nome]', nome_substituto)
        return script

    def _get_script(self, intent_name):
        if intent_name in ['Reagendamento_Objeção_Pessoal', 'Offer_Ex_Aluno', 'Offer_Novo_Aluno', 'Encerramento_Suave', 'Objecao_Preco_Digital_Persistente']:
            return self._format_script(intent_name)
        return self._format_script(intent_name)

    def add_human_touch(self, response: str) -> str:
        if random.random() < 0.25:
            toque = random.choice(FRASES_HUMANIZACAO)
            return f"{response}\n\n[Humanização: {toque}]"
        return response

    def check_lead_history(self, telefone: str) -> Optional[str]:
        if telefone in CRM_INATIVOS and CRM_INATIVOS[telefone]['status'] == 'INATIVO':
            self.nome_lead = CRM_INATIVOS[telefone]['nome']
            return 'Inativo_Identificado'
        return None

    def _simular_nlp(self, message: str) -> str:
        message_lower = message.lower()

        if any(keyword in message_lower for keyword in ['tchau', 'agradeço', 'não vou seguir', 'parar por aqui', 'vou sair', 'não quero de forma alguma']): return 'Recusa_Total'
        if ('preço' in message_lower or 'valor' in message_lower or 'caro' in message_lower): return 'Objecao_Preco_Digital'
        if ('tempo' in message_lower or 'corrido' in message_lower or 'sem tempo' in message_lower): return 'Objecao_Sem_Tempo'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             if 'nome' in message_lower or len(message.split()) <= 4: return 'Resposta_Nome'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            termos_objetivo = ['músculos', 'musculos', 'musculo', 'emagrecer', 'saúde', 'condicionamento', 'foco', 'massa']
            if any(termo in message_lower for termo in termos_objetivo):
                self.contexto_lead['objetivo_pessoal_completo'] = message
                return 'Resposta_Objetivo'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            if any(keyword in message_lower for keyword in ['ex-aluno', 'já fui', 'treinei antes', 'antigo aluno']):
                self.contexto_lead['historico_treino'] = 'EX_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            if any(keyword in message_lower for keyword in ['nunca', 'primeira vez', 'começando agora']):
                self.contexto_lead['historico_treino'] = 'NOVO_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            return 'Resposta_Historico_Treino_Geral'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
             return 'Resposta_Conhece_Academia'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_AGENDAMENTO' or self.contexto_lead['estado_atual'] == 'AGUARDANDO_RESPOSTA_OBJECAO':
             if any(keyword in message_lower for keyword in ['hoje', 'amanhã', 'tarde', 'manhã', 'agora', 'sim', 'quero', 'pode ser']):
                 return 'Agendamento_Sucesso'

        if self.contexto_lead['estado_atual'] == 'INICIO': return 'Interesse_Inicial'

        return 'Intencao_Nao_Identificada'

    # Função auxiliar dedicada para Objeções de Preço
    def _handle_preco_objection(self):

        # Objeção antes do nome (apenas inicia o fluxo normal)
        if self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self.add_human_touch(self._get_script('Interesse_Inicial'))

        self.contexto_lead['objecao_preco_contagem'] += 1

        # Nível 1: Persuasão Pessoal
        if self.contexto_lead['objecao_preco_contagem'] == 1:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self.add_human_touch(self._get_script('Reagendamento_Objeção_Pessoal'))

        # Nível 2: 7 Dias Grátis
        if self.contexto_lead['objecao_preco_contagem'] == 2:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self.add_human_touch(self._get_script('Objecao_Preco_Digital_Persistente'))

        # Nível 3: Transferência
        if self.contexto_lead['objecao_preco_contagem'] >= 3:
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self.add_human_touch(self._get_script('Transferencia_Consultor'))

        # Fallback de segurança (Não deve ser alcançado)
        return self._get_script('Intencao_Nao_Identificada')


    def generate_response(self, client_message: str) -> str:

        if self.contexto_lead['estado_atual'] in ['TRANSFERIDO', 'FIM']:
             return self._get_script('Transferencia_Consultor')

        intent = self._simular_nlp(client_message)

        # 1. TRATAMENTO DE OBJEÇÕES (Funções Auxiliares)
        if intent == 'Objecao_Preco_Digital':
            return self._handle_preco_objection()

        if intent == 'Objecao_Sem_Tempo':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self.add_human_touch(self._get_script(intent))

        # 2. FLUXO DE ENCERRAMENTO (Recusa total)
        if intent == 'Recusa_Total':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self.add_human_touch(self._get_script('Transferencia_Consultor'))

        # 3. FLUXO DE AGENDAMENTO BEM SUCEDIDO
        elif intent == 'Agendamento_Sucesso' and self.contexto_lead['estado_atual'] in ['AGUARDANDO_AGENDAMENTO', 'AGUARDANDO_RESPOSTA_OBJECAO']:
            self.contexto_lead['estado_atual'] = 'FIM'
            return self.add_human_touch(f"Perfeito, {self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'}! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳")

        # -- FLUXO SEQUENCIAL PRINCIPAL --

        # 4. FLUXO DE INTERESSE INICIAL
        elif intent == 'Interesse_Inicial' and self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self.add_human_touch(self._get_script('Interesse_Inicial'))

        # 5. FL

In [ ]:
import pandas as pd
from typing import Dict, Any, Optional
import random

# --- BASES DE CONHECIMENTO ---
SCRIPTS = {
    'Interesse_Inicial': "Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?",
    'Resposta_Objetivo': "Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?",
    'Resposta_Historico_Treino': "E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')",
    'Resposta_Conhece_Academia': "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    'Objecao_Preco_Digital': "Nossa, [Nome]! Temos planos a partir de R$ 49,90. Eu entendo que se sinta assim em relação ao plano, mas está caro comparado a que? Me diz para eu te ajudar a encontrar a melhor opção. 🧐",
    'Objecao_Sem_Tempo': "Super entendo! Por isso mesmo seria ótimo você vir 10 minutinhos só pra conhecer. Posso te mostrar os horários mais tranquilos. Prefere manhã ou tarde?",
    'Inativo_Identificado': "Oi [Nome]! Vi aqui que você já treinou com a gente 💛 Estamos com uma condição especial pra ex-alunos essa semana. Quer que eu te explique?",
    'Transferencia_Consultor': "Certo, [Nome]. Para garantir que todas as suas dúvidas sejam sanadas, vou pedir para uma de nossas consultoras te atender em 5 minutos. Ok?",
    'Intencao_Nao_Identificada': "Me desculpe, ainda não entendi bem. Como consultor, preciso de mais detalhes para te dar a melhor solução."
}
CRM_INATIVOS = {'5511987654321': {'status': 'INATIVO', 'nome': 'Marta'}}
FRASES_HUMANIZACAO = ["Que massa saber disso!", "Você vai gostar muito do nosso ambiente.", "Fico feliz em te ajudar.", "Pode contar comigo."]


# --- CLASSE SalesAgentConsultivo (V33) ---
class SalesAgentConsultivo:

    def __init__(self):
        self.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
        self.nome_lead = ''
        print(f"Agente Consultivo ACADE FINAL (V33 - Objeção Forçada) inicializado.")

    def _format_script(self, script_key: str) -> str:

        if script_key == 'Offer_Novo_Aluno':
             script = "Que ótimo! Para que você veja a qualidade das nossas aulas e o ambiente, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"
        elif script_key == 'Offer_Ex_Aluno':
             script = "Que bom que você pensa em voltar, [Nome]! Desde que você saiu, reformamos tudo e temos equipamentos novos! Para você conferir, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"
        elif script_key == 'Reagendamento_Objeção_Pessoal':
            objetivo_limpo = self.contexto_lead.get('objetivo_pessoal_completo', 'ser mais saudável').strip().rstrip('.')
            script = f"Entendi sua preocupação, [Nome]. Mas você mencionou que o objetivo é **{objetivo_limpo}**. Se você adiar, vai ficar mais difícil atingir essa meta. Vamos agendar sua visita para hoje ou amanhã para garantir que você não perca o foco?"
        elif script_key == 'Encerramento_Suave':
            script = "Que alegria saber que quer começar essa jornada com a gente, [Nome]! Vamos garantir que você alcance seus objetivos. Que tal deixar agendado sua visita para mês que vem? Prefere na primeira semana do mês ou segunda?"
        elif script_key == 'Objecao_Preco_Digital_Persistente':
            script = "Certo, [Nome]! Eu entendo a preocupação! Mas olha só: nossos planos começam a partir de R$ 49,90. Para que você tenha certeza de que é o investimento certo, posso te liberar 7 dias de acesso total ao ambiente e professores, sem compromisso. Topa vir conhecer e testar nosso trabalho?"
        else:
            script = SCRIPTS.get(script_key)

        nome_substituto = self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'
        script = script.replace('[Nome]', nome_substituto)
        return script

    def _get_script(self, intent_name):
        if intent_name in ['Reagendamento_Objeção_Pessoal', 'Offer_Ex_Aluno', 'Offer_Novo_Aluno', 'Encerramento_Suave', 'Objecao_Preco_Digital_Persistente']:
            return self._format_script(intent_name)
        return self._format_script(intent_name)

    def add_human_touch(self, response: str) -> str:
        if random.random() < 0.25:
            toque = random.choice(FRASES_HUMANIZACAO)
            return f"{response}\n\n[Humanização: {toque}]"
        return response

    def check_lead_history(self, telefone: str) -> Optional[str]:
        if telefone in CRM_INATIVOS and CRM_INATIVOS[telefone]['status'] == 'INATIVO':
            self.nome_lead = CRM_INATIVOS[telefone]['nome']
            return 'Inativo_Identificado'
        return None

    def _simular_nlp(self, message: str) -> str:
        message_lower = message.lower()

        if any(keyword in message_lower for keyword in ['tchau', 'agradeço', 'não vou seguir', 'parar por aqui', 'vou sair', 'não quero de forma alguma']): return 'Recusa_Total'
        if ('preço' in message_lower or 'valor' in message_lower or 'caro' in message_lower or 'pesado' in message_lower): return 'Objecao_Preco_Digital'
        if ('tempo' in message_lower or 'corrido' in message_lower or 'sem tempo' in message_lower): return 'Objecao_Sem_Tempo'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             if 'nome' in message_lower or len(message.split()) <= 4: return 'Resposta_Nome'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            termos_objetivo = ['músculos', 'musculos', 'musculo', 'emagrecer', 'saúde', 'condicionamento', 'foco', 'massa']
            if any(termo in message_lower for termo in termos_objetivo):
                self.contexto_lead['objetivo_pessoal_completo'] = message
                return 'Resposta_Objetivo'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            if any(keyword in message_lower for keyword in ['ex-aluno', 'já fui', 'treinei antes', 'antigo aluno']):
                self.contexto_lead['historico_treino'] = 'EX_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            if any(keyword in message_lower for keyword in ['nunca', 'primeira vez', 'começando agora']):
                self.contexto_lead['historico_treino'] = 'NOVO_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            return 'Resposta_Historico_Treino_Geral'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
             return 'Resposta_Conhece_Academia'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_AGENDAMENTO' or self.contexto_lead['estado_atual'] == 'AGUARDANDO_RESPOSTA_OBJECAO':
             if any(keyword in message_lower for keyword in ['hoje', 'amanhã', 'tarde', 'manhã', 'agora', 'sim', 'quero', 'pode ser']):
                 return 'Agendamento_Sucesso'

        if self.contexto_lead['estado_atual'] == 'INICIO': return 'Interesse_Inicial'

        return 'Intencao_Nao_Identificada'

    # Função auxiliar dedicada para Objeções de Preço
    def _handle_preco_objection(self):

        # Objeção antes do nome (apenas inicia o fluxo normal)
        if self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self.add_human_touch(self._get_script('Interesse_Inicial'))

        self.contexto_lead['objecao_preco_contagem'] += 1

        contagem = self.contexto_lead['objecao_preco_contagem']

        if contagem == 1:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self.add_human_touch(self._get_script('Reagendamento_Objeção_Pessoal'))

        elif contagem == 2:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self.add_human_touch(self._get_script('Objecao_Preco_Digital_Persistente'))

        else: # contagem >= 3
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self.add_human_touch(self._get_script('Transferencia_Consultor'))


    def generate_response(self, client_message: str) -> str:

        if self.contexto_lead['estado_atual'] in ['TRANSFERIDO', 'FIM']:
             return self._get_script('Transferencia_Consultor')

        intent = self._simular_nlp(client_message)

        # 1. TRATAMENTO DE OBJEÇÕES (Funções Auxiliares)
        if intent == 'Objecao_Preco_Digital':
            return self._handle_preco_objection()

        if intent == 'Objecao_Sem_Tempo':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self.add_human_touch(self._get_script(intent))

        # 2. FLUXO DE ENCERRAMENTO (Recusa total)
        if intent == 'Recusa_Total':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self.add_human_touch(self._get_script('Transferencia_Consultor'))

        # 3. FLUXO DE AGENDAMENTO BEM SUCEDIDO
        elif intent == 'Agendamento_Sucesso' and self.contexto_lead['estado_atual'] in ['AGUARDANDO_AGENDAMENTO', 'AGUARDANDO_RESPOSTA_OBJECAO']:
            self.contexto_lead['estado_atual'] = 'FIM'
            return self.add_human_touch(f"Perfeito, {self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'}! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳")

        # -- FLUXO SEQUENCIAL PRINCIPAL --

        # 4. FLUXO DE INTERESSE INICIAL
        elif intent == 'Interesse_Inicial' and self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self.add_human_touch(self._get_script('Interesse_Inicial'))

        # 5. FLUXO DE CAPTURA DE NOME
        elif intent == 'Resposta_Nome' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
            self.contexto_lead['objecao_preco_contagem'] = 0
            nome_candidato = client_message.split()[-1].replace('.', '').replace(',', '')
            if len(nome_candidato) > 2 and nome_candidato.lower() not in ['sim', 'não', 'eu', 'ok', 'aqui']: self.nome_lead = nome_candidato

            self.contexto_lead['estado_atual'] = 'AGUARDANDO_OBJETIVO'
            return self._get_script('Resposta_Objetivo')

        # 6. FLUXO DE RESPOSTA OBJETIVO -> AGUARDANDO HISTORICO
        elif intent == 'Resposta_Objetivo' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_HISTORICO'
            return self.add_human_touch(self._get_script('Resposta_Historico_Treino'))

        # 7. FLUXO DE RESPOSTA HISTÓRICO -> CONHECE ACADEMIA
        elif intent in ['Resposta_Historico_Treino_Especifica', 'Resposta_Historico_Treino_Geral'] and self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_VISITA'
            return self.add_human_touch(self._get_script('Resposta_Conhece_Academia'))

        # 8. FLUXO DE RESPOSTA CONHECE ACADEMIA -> OFERTA DE 7 DIAS + AGENDAMENTO
        elif intent == 'Resposta_Conhece_Academia' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_AGENDAMENTO'

            if self.contexto_lead.get('historico_treino') == 'EX_ALUNO':
                 return self.add_human_touch(self._get_script('Offer_Ex_Aluno'))
            else:
                 return self.add_human_touch(self._get_script('Offer_Novo_Aluno'))

        # 9. FALLBACK UNIVERSAL (Se a intenção não foi tratada acima)
        else:
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self.add_human_touch(self._get_script('Transferencia_Consultor'))


# --- FUNÇÕES DE TESTE ---

def testar_fluxo(agente_ia, nome_do_teste: str, mensagens: Dict[str, str]):
    """Função para simular um fluxo de conversa completo."""
    print("=====================================================")
    print(f"✅ INICIANDO TESTE: {nome_do_teste}")
    print("=====================================================")

    agente_ia.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
    agente_ia.nome_lead = ''

    for idx, (cliente_msg, intent_esperada) in enumerate(mensagens.items()):
        agente_response = agente_ia.generate_response(cliente_msg)

        print(f"\n[{idx+1}] CLIENTE: {cliente_msg}")
        print(f"    AGENTE ({agente_ia.contexto_lead['estado_atual']}): {agente_response.splitlines()[0]}...")

        if intent_esperada in agente_response:
             print(f"    [SUCESSO] Contém: '{intent_esperada.splitlines()[0]}'")
        else:
             print(f"    [FALHA] Resposta inesperada. Esperado conter: '{intent_esperada.splitlines()[0]}'")


# --- EXECUÇÃO DOS TESTES FINAIS ---

print("\n--- TESTE DE COBERTURA FINAL (V33) ---")
agente_final = SalesAgentConsultivo()

# 1. FLUXO DE VENDA CONSULTIVA (Persuasão e Ex-Aluno)
fluxo_venda_personalizada = {
    "Olá, queria saber sobre os planos.": "Boa tarde!",
    "Meu nome é Rafaela.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Quero emagrecer porque quero usar biquine final de ano.": "E você já treinou antes ou está começando agora?",
    "Já fui aluno, mas faz 2 anos.": "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    "Eu conheço sim.": "Que bom que você pensa em voltar, Rafaela! Desde que você saiu, reformamos tudo e temos equipamentos novos!",
    "Achei o preço caro.": "Entendi sua preocupação, Rafaela. Mas você mencionou que o objetivo é **Quero emagrecer porque quero usar biquine final de ano**.",
    "Ainda está pesado para mim.": "Certo, Rafaela! Eu entendo a preocupação! Mas olha só: nossos planos começam",
    "Claro que sim, agendamos para amanhã de manhã.": "Perfeito, Rafaela! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳",
}
testar_fluxo(agente_final, "Venda Consultiva (Persuasão e Ex-Aluno)", fluxo_venda_personalizada)

# 2. FLUXO DE NOVO ALUNO COM RECUSA TOTAL
fluxo_novo_aluno = {
    "Tenho interesse na academia.": "Boa tarde!",
    "Meu nome é Pedro.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Quero ganhar massa e ficar forte.": "E você já treinou antes ou está começando agora?",
    "Nunca treinei.": "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    "Não conheço, primeira vez.": "Que ótimo! Para que você veja a qualidade das nossas aulas e o ambiente, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?",
    "Não quero de forma alguma, vou ver outras academias.": "Certo, Pedro. Para garantir que todas as suas dúvidas sejam sanadas, vou pedir para uma de nossas consultoras te atender em 5 minutos. Ok?",
}
testar_fluxo(agente_final, "Novo Aluno e Recusa Total (Transferência)", fluxo_novo_aluno)


--- TESTE DE COBERTURA FINAL (V33) ---
Agente Consultivo ACADE FINAL (V33 - Objeção Forçada) inicializado.
✅ INICIANDO TESTE: Venda Consultiva (Persuasão e Ex-Aluno)

[1] CLIENTE: Olá, queria saber sobre os planos.
    AGENTE (AGUARDANDO_NOME): Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?...
    [SUCESSO] Contém: 'Boa tarde!'

[2] CLIENTE: Meu nome é Rafaela.
    AGENTE (AGUARDANDO_OBJETIVO): Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?...
    [SUCESSO] Contém: 'Pra eu te orientar direitinho, qual é o seu objetivo hoje?'

[3] CLIENTE: Quero emagrecer porque quero usar biquine final de ano.
    AGENTE (AGUARDANDO_HISTORICO): E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')...
    [SUCESSO] Contém: 'E você já treinou antes ou está começando agora?'

[4] CLIENTE: Já fui aluno, mas faz 2 a

In [ ]:
import pandas as pd
from typing import Dict, Any, Optional
import random

# --- BASES DE CONHECIMENTO ---
SCRIPTS = {
    'Interesse_Inicial': "Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?",
    'Resposta_Objetivo': "Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?",
    'Resposta_Historico_Treino': "E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')",
    'Resposta_Conhece_Academia': "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    'Objecao_Preco_Digital': "Nossa, [Nome]! Temos planos a partir de R$ 49,90. Eu entendo que se sinta assim em relação ao plano, mas está caro comparado a que? Me diz para eu te ajudar a encontrar a melhor opção. 🧐",
    'Objecao_Sem_Tempo': "Super entendo! Por isso mesmo seria ótimo você vir 10 minutinhos só pra conhecer. Posso te mostrar os horários mais tranquilos. Prefere manhã ou tarde?",
    'Inativo_Identificado': "Oi [Nome]! Vi aqui que você já treinou com a gente 💛 Estamos com uma condição especial pra ex-alunos essa semana. Quer que eu te explique?",

    # SCRIPTS V34
    'Agendamento_Avaliacao': "Claro! Para a avaliação física, vou te transferir agora para nossa equipe técnica. Ela já verifica sua disponibilidade e agenda o melhor horário. Posso transferir em 1 minuto, [Nome]?",
    'Interesse_Suplementos': "Vendemos sim! Para te dar o preço e ver o estoque exato (temos novidades!), vou te transferir para o nosso setor de vendas. Eles te atendem em 1 minuto. Tudo bem, [Nome]?",

    'Transferencia_Consultor': "Certo, [Nome]. Para garantir que todas as suas dúvidas sejam sanadas, vou pedir para uma de nossas consultoras te atender em 5 minutos. Ok?",
    'Intencao_Nao_Identificada': "Me desculpe, ainda não entendi bem. Como consultor, preciso de mais detalhes para te dar a melhor solução."
}
CRM_INATIVOS = {'5511987654321': {'status': 'INATIVO', 'nome': 'Marta'}}
FRASES_HUMANIZACAO = ["Que massa saber disso!", "Você vai gostar muito do nosso ambiente.", "Fico feliz em te ajudar.", "Pode contar comigo."]


# --- CLASSE SalesAgentConsultivo (V40) ---
class SalesAgentConsultivo:

    def __init__(self):
        self.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
        self.nome_lead = ''
        print(f"Agente Consultivo ACADE FINAL (V40 - Correção Extração Nome) inicializado.")

    def _format_script(self, script_key: str) -> str:

        if script_key == 'Offer_Novo_Aluno':
             script = "Que ótimo! Para que você veja a qualidade das nossas aulas e o ambiente, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"
        elif script_key == 'Offer_Ex_Aluno':
             script = "Que bom que você pensa em voltar, [Nome]! Desde que você saiu, reformamos tudo e temos equipamentos novos! Para você conferir, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"
        elif script_key == 'Reagendamento_Objeção_Pessoal':
            objetivo_limpo = self.contexto_lead.get('objetivo_pessoal_completo', 'ser mais saudável').strip().rstrip('.')
            script = f"Entendi sua preocupação, [Nome]. Mas você mencionou que o objetivo é **{objetivo_limpo}**. Se você adiar, vai ficar mais difícil atingir essa meta. Vamos agendar sua visita para hoje ou amanhã para garantir que você não perca o foco?"
        elif script_key == 'Encerramento_Suave':
            script = "Que alegria saber que quer começar essa jornada com a gente, [Nome]! Vamos garantir que você alcance seus objetivos. Que tal deixar agendado sua visita para mês que vem? Prefere na primeira semana do mês ou segunda?"
        elif script_key == 'Objecao_Preco_Digital_Persistente':
            script = "Certo, [Nome]! Eu entendo a preocupação! Mas olha só: nossos planos começam a partir de R$ 49,90. Para que você tenha certeza de que é o investimento certo, posso te liberar 7 dias de acesso total ao ambiente e professores, sem compromisso. Topa vir conhecer e testar nosso trabalho?"
        else:
            script = SCRIPTS.get(script_key)

        nome_substituto = self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'
        script = script.replace('[Nome]', nome_substituto)
        return script

    def _get_script(self, intent_name):
        script = self._format_script(intent_name)

        # Não adiciona humanização se for uma transferência imediata ou se for o pedido inicial de nome
        if intent_name in ['Agendamento_Avaliacao', 'Interesse_Suplementos', 'Transferencia_Consultor', 'Interesse_Inicial']:
            return script

        return self.add_human_touch(script)


    def add_human_touch(self, response: str) -> str:
        if random.random() < 0.25:
            toque = random.choice(FRASES_HUMANIZACAO)
            return f"{response}\n\n[Humanização: {toque}]"
        return response

    def check_lead_history(self, telefone: str) -> Optional[str]:
        if telefone in CRM_INATIVOS and CRM_INATIVOS[telefone]['status'] == 'INATIVO':
            self.nome_lead = CRM_INATIVOS[telefone]['nome']
            return 'Inativo_Identificado'
        return None

    def _simular_nlp(self, message: str) -> str:
        message_lower = message.lower()

        # --- ISOLAMENTO DE CAPTURA DE NOME (V39) ---
        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             # Regra rígida para Resposta_Nome: Contém "nome" ou é frase curta.
             if 'nome' in message_lower or len(message.split()) <= 4:
                 return 'Resposta_Nome'
             else:
                 return 'Intencao_Nao_Identificada'
        # ---------------------------------------------

        # INTENÇÕES DE ALTA PRIORIDADE (Transversais - Verificadas APENAS APÓS AGUARDANDO_NOME)
        if any(keyword in message_lower for keyword in ['avaliação física', 'agendar avaliação', 'como funciona avaliação']): return 'Agendamento_Avaliacao'
        if any(keyword in message_lower for keyword in ['suplementos', 'whey', 'creatina', 'comprar whey', 'vendem suplementos']): return 'Interesse_Suplementos'
        if any(keyword in message_lower for keyword in ['tchau', 'agradeço', 'não vou seguir', 'parar por aqui', 'vou sair', 'não quero de forma alguma']): return 'Recusa_Total'
        if ('preço' in message_lower or 'valor' in message_lower or 'caro' in message_lower or 'pesado' in message_lower): return 'Objecao_Preco_Digital'
        if ('tempo' in message_lower or 'corrido' in message_lower or 'sem tempo' in message_lower): return 'Objecao_Sem_Tempo'

        # INTENÇÕES DE FLUXO SEQUENCIAL (Baseadas em Estado)

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            termos_objetivo = ['músculos', 'musculos', 'musculo', 'emagrecer', 'saúde', 'condicionamento', 'foco', 'massa']
            if any(termo in message_lower for termo in termos_objetivo):
                self.contexto_lead['objetivo_pessoal_completo'] = message
                return 'Resposta_Objetivo'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            if any(keyword in message_lower for keyword in ['ex-aluno', 'já fui', 'treinei antes', 'antigo aluno']):
                self.contexto_lead['historico_treino'] = 'EX_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            if any(keyword in message_lower for keyword in ['nunca', 'primeira vez', 'começando agora']):
                self.contexto_lead['historico_treino'] = 'NOVO_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            return 'Resposta_Historico_Treino_Geral'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
             return 'Resposta_Conhece_Academia'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_AGENDAMENTO' or self.contexto_lead['estado_atual'] == 'AGUARDANDO_RESPOSTA_OBJECAO':
             if any(keyword in message_lower for keyword in ['hoje', 'amanhã', 'tarde', 'manhã', 'agora', 'sim', 'quero', 'pode ser']):
                 return 'Agendamento_Sucesso'

        if self.contexto_lead['estado_atual'] == 'INICIO': return 'Interesse_Inicial'

        return 'Intencao_Nao_Identificada'

    def _handle_preco_objection(self):

        if self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self._get_script('Interesse_Inicial')

        self.contexto_lead['objecao_preco_contagem'] += 1
        contagem = self.contexto_lead['objecao_preco_contagem']

        if contagem == 1:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self._get_script('Reagendamento_Objeção_Pessoal')

        elif contagem == 2:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self._get_script('Objecao_Preco_Digital_Persistente')

        else: # contagem >= 3
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script('Transferencia_Consultor')


    def generate_response(self, client_message: str) -> str:

        if self.contexto_lead['estado_atual'] in ['TRANSFERIDO', 'FIM']:
             return self._get_script('Transferencia_Consultor')

        intent = self._simular_nlp(client_message)

        # 1. TRATAMENTO DE INTERESSE INICIAL (Máxima Prioridade)
        if self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self._get_script('Interesse_Inicial')

        # 2. CAPTURA DE NOME (Prioridade Máxima no estado AGUARDANDO_NOME)
        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             if intent == 'Resposta_Nome':
                 self.contexto_lead['objecao_preco_contagem'] = 0

                 # --- CORREÇÃO V40: Extração de nome sempre pelo último token ---
                 tokens = client_message.split()
                 nome_candidato = tokens[-1].replace('.', '').replace(',', '')
                 # ----------------------------------------------------------------

                 if len(nome_candidato) > 2 and nome_candidato.lower() not in ['sim', 'não', 'eu', 'ok', 'aqui']:
                     self.nome_lead = nome_candidato

                 self.contexto_lead['estado_atual'] = 'AGUARDANDO_OBJETIVO'
                 return self._get_script('Resposta_Objetivo')

             # Se a intenção não for Resposta_Nome, repete o pedido.
             return self._get_script('Interesse_Inicial')


        # 3. TRATAMENTO DE INTENÇÕES IMEDIATAS (Avaliação e Suplementos) - Apenas após a captura de nome
        if intent in ['Agendamento_Avaliacao', 'Interesse_Suplementos']:
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script(intent)

        # 4. TRATAMENTO DE OBJEÇÕES DE VENDA
        if intent == 'Objecao_Preco_Digital':
            return self._handle_preco_objection()

        if intent == 'Objecao_Sem_Tempo':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script(intent)

        # 5. FLUXO DE ENCERRAMENTO (Recusa total)
        if intent == 'Recusa_Total':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script('Transferencia_Consultor')

        # 6. FLUXO DE AGENDAMENTO BEM SUCEDIDO
        elif intent == 'Agendamento_Sucesso' and self.contexto_lead['estado_atual'] in ['AGUARDANDO_AGENDAMENTO', 'AGUARDANDO_RESPOSTA_OBJECAO']:
            self.contexto_lead['estado_atual'] = 'FIM'
            return f"Perfeito, {self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'}! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳"

        # 7. FLUXO SEQUENCIAL PRINCIPAL (Objetivo, Histórico, Visita)

        elif intent == 'Resposta_Objetivo' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_HISTORICO'
            return self._get_script('Resposta_Historico_Treino')

        elif intent in ['Resposta_Historico_Treino_Especifica', 'Resposta_Historico_Treino_Geral'] and self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_VISITA'
            return self._get_script('Resposta_Conhece_Academia')

        elif intent == 'Resposta_Conhece_Academia' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_AGENDAMENTO'

            if self.contexto_lead.get('historico_treino') == 'EX_ALUNO':
                 return self._get_script('Offer_Ex_Aluno')
            else:
                 return self._get_script('Offer_Novo_Aluno')

        # 8. FALLBACK UNIVERSAL
        else:
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script('Transferencia_Consultor')


# --- FUNÇÕES DE TESTE ---

def testar_fluxo(agente_ia, nome_do_teste: str, mensagens: Dict[str, str]):
    """Função para simular um fluxo de conversa completo."""
    print("=====================================================")
    print(f"✅ INICIANDO TESTE: {nome_do_teste}")
    print("=====================================================")

    agente_ia.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
    agente_ia.nome_lead = ''

    for idx, (cliente_msg, intent_esperada) in enumerate(mensagens.items()):
        agente_response_full = agente_ia.generate_response(cliente_msg)
        agente_response = agente_response_full.splitlines()[0] if agente_response_full else ''

        print(f"\n[{idx+1}] CLIENTE: {cliente_msg}")
        print(f"    AGENTE ({agente_ia.contexto_lead['estado_atual']}): {agente_response}...")

        # O teste verifica a string original, que pode incluir a humanização.
        if intent_esperada in agente_response_full:
             print(f"    [SUCESSO] Contém: '{intent_esperada.splitlines()[0]}'")
        else:
             print(f"    [FALHA] Resposta inesperada. Esperado conter: '{intent_esperada.splitlines()[0]}'")


# --- EXECUÇÃO DOS TESTES FINAIS ---

print("\n--- TESTE DE COBERTURA FINAL (V40) ---")
agente_final = SalesAgentConsultivo()

# 1. FLUXO DE VENDA CONSULTIVA (V33 - Baseline)
fluxo_venda_personalizada = {
    "Olá, queria saber sobre os planos.": "Boa tarde!",
    "Meu nome é Rafaela.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Quero emagrecer porque quero usar biquine final de ano.": "E você já treinou antes ou está começando agora?",
    "Já fui aluno, mas faz 2 anos.": "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    "Eu conheço sim.": "Que bom que você pensa em voltar, Rafaela! Desde que você saiu, reformamos tudo e temos equipamentos novos! Para você conferir, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?",
    "Achei o preço caro.": "Entendi sua preocupação, Rafaela. Mas você mencionou que o objetivo é **Quero emagrecer porque quero usar biquine final de ano**.",
    "Ainda está pesado para mim.": "Certo, Rafaela! Eu entendo a preocupação! Mas olha só: nossos planos começam",
    "Claro que sim, agendamos para amanhã de manhã.": "Perfeito, Rafaela! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳",
}
testar_fluxo(agente_final, "Venda Consultiva (Baseline V33)", fluxo_venda_personalizada)

# 2. NOVO FLUXO - AGENDAMENTO DE AVALIAÇÃO (Intenção Imediata)
fluxo_avaliacao = {
    "Olá, quero agendar avaliação física.": "Boa tarde!", # ESTADO INICIO -> AGUARDANDO_NOME
    "Meu nome é Carol.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?", # ESTADO AGUARDANDO_NOME -> AGUARDANDO_OBJETIVO
    "Quero agendar avaliação física": "Claro! Para a avaliação física, vou te transferir agora para nossa equipe técnica. Ela já verifica sua disponibilidade e agenda o melhor horário. Posso transferir em 1 minuto, Carol?", # ESTADO AGUARDANDO_OBJETIVO -> TRANSFERIDO
}
testar_fluxo(agente_final, "Fluxo 2: Avaliação Física", fluxo_avaliacao)

# 3. NOVO FLUXO - INTERESSE EM SUPLEMENTOS (Intenção Imediata)
fluxo_suplementos = {
    "Oi, vocês vendem suplementos?": "Boa tarde!", # ESTADO INICIO -> AGUARDANDO_NOME
    "Sou o Marcelo.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?", # ESTADO AGUARDANDO_NOME -> AGUARDANDO_OBJETIVO
    "Queria comprar whey protein.": "Vendemos sim! Para te dar o preço e ver o estoque exato (temos novidades!), vou te transferir para o nosso setor de vendas. Eles te atendem em 1 minuto. Tudo bem, Marcelo?", # ESTADO AGUARDANDO_OBJETIVO -> TRANSFERIDO
}
testar_fluxo(agente_final, "Fluxo 3: Compra de Suplementos", fluxo_suplementos)


--- TESTE DE COBERTURA FINAL (V40) ---
Agente Consultivo ACADE FINAL (V40 - Correção Extração Nome) inicializado.
✅ INICIANDO TESTE: Venda Consultiva (Baseline V33)

[1] CLIENTE: Olá, queria saber sobre os planos.
    AGENTE (AGUARDANDO_NOME): Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?...
    [SUCESSO] Contém: 'Boa tarde!'

[2] CLIENTE: Meu nome é Rafaela.
    AGENTE (AGUARDANDO_OBJETIVO): Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?...
    [SUCESSO] Contém: 'Pra eu te orientar direitinho, qual é o seu objetivo hoje?'

[3] CLIENTE: Quero emagrecer porque quero usar biquine final de ano.
    AGENTE (AGUARDANDO_HISTORICO): E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')...
    [SUCESSO] Contém: 'E você já treinou antes ou está começando agora?'

[4] CLIENTE: Já fui aluno, mas faz 2 an

In [ ]:
import pandas as pd
from typing import Dict, Any, Optional
import random

# --- BASES DE CONHECIMENTO ---
SCRIPTS = {
    'Interesse_Inicial': "Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?",
    'Resposta_Objetivo': "Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?",
    'Resposta_Historico_Treino': "E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')",
    'Resposta_Conhece_Academia': "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    'Objecao_Preco_Digital': "Nossa, [Nome]! Temos planos a partir de R$ 49,90. Eu entendo que se sinta assim em relação ao plano, mas está caro comparado a que? Me diz para eu te ajudar a encontrar a melhor opção. 🧐",
    'Objecao_Sem_Tempo': "Super entendo! Por isso mesmo seria ótimo você vir 10 minutinhos só pra conhecer. Posso te mostrar os horários mais tranquilos. Prefere manhã ou tarde?",
    'Inativo_Identificado': "Oi [Nome]! Vi aqui que você já treinou com a gente 💛 Estamos com uma condição especial pra ex-alunos essa semana. Quer que eu te explique?",

    # SCRIPTS V34
    'Agendamento_Avaliacao': "Claro! Para a avaliação física, vou te transferir agora para nossa equipe técnica. Ela já verifica sua disponibilidade e agenda o melhor horário. Posso transferir em 1 minuto, [Nome]?",
    'Interesse_Suplementos': "Vendemos sim! Para te dar o preço e ver o estoque exato (temos novidades!), vou te transferir para o nosso setor de vendas. Eles te atendem em 1 minuto. Tudo bem, [Nome]?",

    'Transferencia_Consultor': "Certo, [Nome]. Para garantir que todas as suas dúvidas sejam sanadas, vou pedir para uma de nossas consultoras te atender em 5 minutos. Ok?",
    'Intencao_Nao_Identificada': "Me desculpe, ainda não entendi bem. Como consultor, preciso de mais detalhes para te dar a melhor solução."
}
CRM_INATIVOS = {'5511987654321': {'status': 'INATIVO', 'nome': 'Marta'}}
FRASES_HUMANIZACAO = ["Que massa saber disso!", "Você vai gostar muito do nosso ambiente.", "Fico feliz em te ajudar.", "Pode contar comigo."]


# --- CLASSE SalesAgentConsultivo (V41) ---
class SalesAgentConsultivo:

    def __init__(self):
        self.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
        self.nome_lead = ''
        print(f"Agente Consultivo ACADE FINAL (V41 - Prioridade de Fechamento) inicializado.")

    def _format_script(self, script_key: str) -> str:

        if script_key == 'Offer_Novo_Aluno':
             # Usando aspas triplas para strings longas com aspas duplas internas
             script = """Que ótimo! Para que você veja a qualidade das nossas aulas e o ambiente, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"""
        elif script_key == 'Offer_Ex_Aluno':
             # Usando aspas triplas
             script = """Que bom que você pensa em voltar, [Nome]! Desde que você saiu, reformamos tudo e temos equipamentos novos! Para você conferir, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"""
        elif script_key == 'Reagendamento_Objeção_Pessoal':
            objetivo_limpo = self.contexto_lead.get('objetivo_pessoal_completo', 'ser mais saudável').strip().rstrip('.')
            script = f"Entendi sua preocupação, [Nome]. Mas você mencionou que o objetivo é **{objetivo_limpo}**. Se você adiar, vai ficar mais difícil atingir essa meta. Vamos agendar sua visita para hoje ou amanhã para garantir que você não perca o foco?"
        elif script_key == 'Encerramento_Suave':
            script = "Que alegria saber que quer começar essa jornada com a gente, [Nome]! Vamos garantir que você alcance seus objetivos. Que tal deixar agendado sua visita para mês que vem? Prefere na primeira semana do mês ou segunda?"
        elif script_key == 'Objecao_Preco_Digital_Persistente':
            script = "Certo, [Nome]! Eu entendo a preocupação! Mas olha só: nossos planos começam a partir de R$ 49,90. Para que você tenha certeza de que é o investimento certo, posso te liberar 7 dias de acesso total ao ambiente e professores, sem compromisso. Topa vir conhecer e testar nosso trabalho?"
        else:
            script = SCRIPTS.get(script_key)

        nome_substituto = self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'
        script = script.replace('[Nome]', nome_substituto)
        return script

    def _get_script(self, intent_name):
        script = self._format_script(intent_name)

        # Não adiciona humanização se for uma transferência imediata ou se for o pedido inicial de nome
        if intent_name in ['Agendamento_Avaliacao', 'Interesse_Suplementos', 'Transferencia_Consultor', 'Interesse_Inicial']:
            return script

        return self.add_human_touch(script)


    def add_human_touch(self, response: str) -> str:
        if random.random() < 0.25:
            toque = random.choice(FRASES_HUMANIZACAO)
            return f"{response}\n\n[Humanização: {toque}]"
        return response

    def check_lead_history(self, telefone: str) -> Optional[str]:
        if telefone in CRM_INATIVOS and CRM_INATIVOS[telefone]['status'] == 'INATIVO':
            self.nome_lead = CRM_INATIVOS[telefone]['nome']
            return 'Inativo_Identificado'
        return None

    def _simular_nlp(self, message: str) -> str:
        message_lower = message.lower()

        # --- 1. AÇÃO CORREÇÃO V41: Prioridade de Fechamento Contextual ---
        if self.contexto_lead['estado_atual'] in ['AGUARDANDO_AGENDAMENTO', 'AGUARDANDO_RESPOSTA_OBJECAO']:
             if any(keyword in message_lower for keyword in ['hoje', 'amanhã', 'tarde', 'manhã', 'agora', 'sim', 'quero', 'pode ser', 'consigo ir']):
                 return 'Agendamento_Sucesso'
        # -------------------------------------------------------------------

        # --- 2. ISOLAMENTO DE CAPTURA DE NOME (V39) ---
        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             # Regra rígida para Resposta_Nome: Contém "nome" ou é frase curta.
             if 'nome' in message_lower or len(message.split()) <= 4:
                 return 'Resposta_Nome'
             else:
                 return 'Intencao_Nao_Identificada'
        # ---------------------------------------------

        # --- 3. INTENÇÕES DE ALTA PRIORIDADE (Transversais) ---
        if any(keyword in message_lower for keyword in ['avaliação física', 'agendar avaliação', 'como funciona avaliação']): return 'Agendamento_Avaliacao'
        if any(keyword in message_lower for keyword in ['suplementos', 'whey', 'creatina', 'comprar whey', 'vendem suplementos']): return 'Interesse_Suplementos'
        if any(keyword in message_lower for keyword in ['tchau', 'agradeço', 'não vou seguir', 'parar por aqui', 'vou sair', 'não quero de forma alguma']): return 'Recusa_Total'
        if ('preço' in message_lower or 'valor' in message_lower or 'caro' in message_lower or 'pesado' in message_lower): return 'Objecao_Preco_Digital'
        if ('tempo' in message_lower or 'corrido' in message_lower or 'sem tempo' in message_lower): return 'Objecao_Sem_Tempo'

        # --- 4. INTENÇÕES DE FLUXO SEQUENCIAL ---

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            termos_objetivo = ['músculos', 'musculos', 'musculo', 'emagrecer', 'saúde', 'condicionamento', 'foco', 'massa']
            if any(termo in message_lower for termo in termos_objetivo):
                self.contexto_lead['objetivo_pessoal_completo'] = message
                return 'Resposta_Objetivo'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            if any(keyword in message_lower for keyword in ['ex-aluno', 'já fui', 'treinei antes', 'antigo aluno']):
                self.contexto_lead['historico_treino'] = 'EX_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            if any(keyword in message_lower for keyword in ['nunca', 'primeira vez', 'começando agora']):
                self.contexto_lead['historico_treino'] = 'NOVO_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            return 'Resposta_Historico_Treino_Geral'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
             return 'Resposta_Conhece_Academia'

        if self.contexto_lead['estado_atual'] == 'INICIO': return 'Interesse_Inicial'

        return 'Intencao_Nao_Identificada'

    def _handle_preco_objection(self):

        if self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self._get_script('Interesse_Inicial')

        self.contexto_lead['objecao_preco_contagem'] += 1
        contagem = self.contexto_lead['objecao_preco_contagem']

        if contagem == 1:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self._get_script('Reagendamento_Objeção_Pessoal')

        elif contagem == 2:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self._get_script('Objecao_Preco_Digital_Persistente')

        else: # contagem >= 3
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script('Transferencia_Consultor')


    def generate_response(self, client_message: str) -> str:

        if self.contexto_lead['estado_atual'] in ['TRANSFERIDO', 'FIM']:
             return self._get_script('Transferencia_Consultor')

        intent = self._simular_nlp(client_message)

        # 1. TRATAMENTO DE INTERESSE INICIAL (Máxima Prioridade)
        if self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self._get_script('Interesse_Inicial')

        # 2. CAPTURA DE NOME (Prioridade Máxima no estado AGUARDANDO_NOME)
        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             if intent == 'Resposta_Nome':
                 self.contexto_lead['objecao_preco_contagem'] = 0

                 # Extração de nome (V40): sempre pelo último token
                 tokens = client_message.split()
                 nome_candidato = tokens[-1].replace('.', '').replace(',', '')

                 if len(nome_candidato) > 2 and nome_candidato.lower() not in ['sim', 'não', 'eu', 'ok', 'aqui']:
                     self.nome_lead = nome_candidato

                 self.contexto_lead['estado_atual'] = 'AGUARDANDO_OBJETIVO'
                 return self._get_script('Resposta_Objetivo')

             # Se a intenção não for Resposta_Nome, repete o pedido.
             return self._get_script('Interesse_Inicial')

        # 3. FLUXO DE AGENDAMENTO BEM SUCEDIDO
        if intent == 'Agendamento_Sucesso':
            self.contexto_lead['estado_atual'] = 'FIM'
            return f"Perfeito, {self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'}! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳"

        # 4. TRATAMENTO DE INTENÇÕES IMEDIATAS (Avaliação e Suplementos) - Apenas após a captura de nome
        if intent in ['Agendamento_Avaliacao', 'Interesse_Suplementos']:
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script(intent)

        # 5. TRATAMENTO DE OBJEÇÕES DE VENDA
        if intent == 'Objecao_Preco_Digital':
            return self._handle_preco_objection()

        if intent == 'Objecao_Sem_Tempo':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script(intent)

        # 6. FLUXO DE ENCERRAMENTO (Recusa total)
        if intent == 'Recusa_Total':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script('Transferencia_Consultor')

        # 7. FLUXO SEQUENCIAL PRINCIPAL (Objetivo, Histórico, Visita)

        elif intent == 'Resposta_Objetivo' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_HISTORICO'
            return self._get_script('Resposta_Historico_Treino')

        elif intent in ['Resposta_Historico_Treino_Especifica', 'Resposta_Historico_Treino_Geral'] and self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_VISITA'
            return self._get_script('Resposta_Conhece_Academia')

        elif intent == 'Resposta_Conhece_Academia' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_AGENDAMENTO'

            if self.contexto_lead.get('historico_treino') == 'EX_ALUNO':
                 return self._get_script('Offer_Ex_Aluno')
            else:
                 return self._get_script('Offer_Novo_Aluno')

        # 8. FALLBACK UNIVERSAL
        else:
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script('Transferencia_Consultor')


# --- FUNÇÕES DE TESTE ---

def testar_fluxo(agente_ia, nome_do_teste: str, mensagens: Dict[str, str]):
    """Função para simular um fluxo de conversa completo."""
    print("=====================================================")
    print(f"✅ INICIANDO TESTE: {nome_do_teste}")
    print("=====================================================")

    agente_ia.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
    agente_ia.nome_lead = ''

    for idx, (cliente_msg, intent_esperada) in enumerate(mensagens.items()):
        agente_response_full = agente_ia.generate_response(cliente_msg)
        agente_response = agente_response_full.splitlines()[0] if agente_response_full else ''

        print(f"\n[{idx+1}] CLIENTE: {cliente_msg}")
        print(f"    AGENTE ({agente_ia.contexto_lead['estado_atual']}): {agente_response}...")

        if intent_esperada in agente_response_full:
             print(f"    [SUCESSO] Contém: '{intent_esperada.splitlines()[0]}'")
        else:
             print(f"    [FALHA] Resposta inesperada. Esperado conter: '{intent_esperada.splitlines()[0]}'")


# --- FLUXOS DE TESTE ---

# 1. FLUXO DE VENDA CONSULTIVA (Baseline V33)
fluxo_venda_personalizada = {
    "Olá, queria saber sobre os planos.": "Boa tarde!",
    "Meu nome é Rafaela.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Quero emagrecer porque quero usar biquine final de ano.": "E você já treinou antes ou está começando agora?",
    "Já fui aluno, mas faz 2 anos.": "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    "Eu conheço sim.": "Que bom que você pensa em voltar, Rafaela! Desde que você saiu, reformamos tudo e temos equipamentos novos! Para você conferir, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?",
    "Achei o preço caro.": "Entendi sua preocupação, Rafaela. Mas você mencionou que o objetivo é **Quero emagrecer porque quero usar biquine final de ano**.",
    "Ainda está pesado para mim.": "Certo, Rafaela! Eu entendo a preocupação! Mas olha só: nossos planos começam",
    "Claro que sim, agendamos para amanhã de manhã.": "Perfeito, Rafaela! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳",
}

# 2. NOVO FLUXO - AGENDAMENTO DE AVALIAÇÃO (Intenção Imediata)
fluxo_avaliacao = {
    "Olá, quero agendar avaliação física.": "Boa tarde!",
    "Meu nome é Carol.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Quero agendar avaliação física": "Claro! Para a avaliação física, vou te transferir agora para nossa equipe técnica. Ela já verifica sua disponibilidade e agenda o melhor horário. Posso transferir em 1 minuto, Carol?",
}

# 3. NOVO FLUXO - INTERESSE EM SUPLEMENTOS (Intenção Imediata)
fluxo_suplementos = {
    "Oi, vocês vendem suplementos?": "Boa tarde!",
    "Sou o Marcelo.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Queria comprar whey protein.": "Vendemos sim! Para te dar o preço e ver o estoque exato (temos novidades!), vou te transferir para o nosso setor de vendas. Eles te atendem em 1 minuto. Tudo bem, Marcelo?",
}

# 4. TESTE DE FLUXO WHATSAPP (Resistência Inicial)
fluxo_whatsapp = {
    "Qual o valor dos planos?": "Boa tarde!",
    "Me chamo João.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?",
    "Ganhar massa, preciso crescer.": "E você já treinou antes ou está começando agora?",
    "Começando agora.": "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    # Corrigido para aspas triplas para evitar problemas de string literal.
    "Nunca fui.": "Que ótimo! Para que você veja a qualidade das nossas aulas e o ambiente, vou te liberar um **passe de 7 dias grátis**",
    "Hoje tá corrido, mas amanhã eu consigo ir.": "Perfeito, João! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳",
}


# --- EXECUÇÃO DOS TESTES FINAIS ---

print("\n--- TESTE DE COBERTURA FINAL (V41) ---")
agente_final = SalesAgentConsultivo()

# Testes Originais
testar_fluxo(agente_final, "Venda Consultiva (Baseline V33)", fluxo_venda_personalizada)
agente_final = SalesAgentConsultivo()
testar_fluxo(agente_final, "Fluxo 2: Avaliação Física", fluxo_avaliacao)
agente_final = SalesAgentConsultivo()
testar_fluxo(agente_final, "Fluxo 3: Compra de Suplementos", fluxo_suplementos)

# Novo Teste do WhatsApp
agente_final = SalesAgentConsultivo()
testar_fluxo(agente_final, "Teste 4: Resistência Inicial (Preço) + Conversão", fluxo_whatsapp)


--- TESTE DE COBERTURA FINAL (V41) ---
Agente Consultivo ACADE FINAL (V41 - Prioridade de Fechamento) inicializado.
✅ INICIANDO TESTE: Venda Consultiva (Baseline V33)

[1] CLIENTE: Olá, queria saber sobre os planos.
    AGENTE (AGUARDANDO_NOME): Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?...
    [SUCESSO] Contém: 'Boa tarde!'

[2] CLIENTE: Meu nome é Rafaela.
    AGENTE (AGUARDANDO_OBJETIVO): Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?...
    [SUCESSO] Contém: 'Pra eu te orientar direitinho, qual é o seu objetivo hoje?'

[3] CLIENTE: Quero emagrecer porque quero usar biquine final de ano.
    AGENTE (AGUARDANDO_HISTORICO): E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')...
    [SUCESSO] Contém: 'E você já treinou antes ou está começando agora?'

[4] CLIENTE: Já fui aluno, mas faz 2 

In [ ]:
#teste de robustez

In [ ]:
import pandas as pd
from typing import Dict, Any, Optional
import random

# --- BASES DE CONHECIMENTO ---
SCRIPTS = {
    'Interesse_Inicial': "Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?",
    'Resposta_Objetivo': "Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?",
    'Resposta_Historico_Treino': "E você já treinou antes ou está começando agora? (Responda: 'Nunca treinei' ou 'Já fui aluno')",
    'Resposta_Conhece_Academia': "Entendi! E você já conhece nossa academia por dentro ou seria sua primeira vez aqui?",
    'Objecao_Preco_Digital': "Nossa, [Nome]! Temos planos a partir de R$ 49,90. Eu entendo que se sinta assim em relação ao plano, mas está caro comparado a que? Me diz para eu te ajudar a encontrar a melhor opção. 🧐",
    'Objecao_Sem_Tempo': "Super entendo! Por isso mesmo seria ótimo você vir 10 minutinhos só pra conhecer. Posso te mostrar os horários mais tranquilos. Prefere manhã ou tarde?",
    'Inativo_Identificado': "Oi [Nome]! Vi aqui que você já treinou com a gente 💛 Estamos com uma condição especial pra ex-alunos essa semana. Quer que eu te explique?",

    # SCRIPTS V34
    'Agendamento_Avaliacao': "Claro! Para a avaliação física, vou te transferir agora para nossa equipe técnica. Ela já verifica sua disponibilidade e agenda o melhor horário. Posso transferir em 1 minuto, [Nome]?",
    'Interesse_Suplementos': "Vendemos sim! Para te dar o preço e ver o estoque exato (temos novidades!), vou te transferir para o nosso setor de vendas. Eles te atendem em 1 minuto. Tudo bem, [Nome]?",

    'Transferencia_Consultor': "Certo, [Nome]. Para garantir que todas as suas dúvidas sejam sanadas, vou pedir para uma de nossas consultoras te atender em 5 minutos. Ok?",
    'Intencao_Nao_Identificada': "Me desculpe, ainda não entendi bem. Como consultor, preciso de mais detalhes para te dar a melhor solução."
}
CRM_INATIVOS = {'5511987654321': {'status': 'INATIVO', 'nome': 'Marta'}}
FRASES_HUMANIZACAO = ["Que massa saber disso!", "Você vai gostar muito do nosso ambiente.", "Fico feliz em te ajudar.", "Pode contar comigo."]


# --- CLASSE SalesAgentConsultivo (V42) ---
class SalesAgentConsultivo:

    def __init__(self):
        self.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
        self.nome_lead = ''
        print(f"Agente Consultivo ACADE FINAL (V42 - Teste de Borda de Prioridade) inicializado.")

    def _format_script(self, script_key: str) -> str:

        if script_key == 'Offer_Novo_Aluno':
             script = """Que ótimo! Para que você veja a qualidade das nossas aulas e o ambiente, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"""
        elif script_key == 'Offer_Ex_Aluno':
             script = """Que bom que você pensa em voltar, [Nome]! Desde que você saiu, reformamos tudo e temos equipamentos novos! Para você conferir, vou te liberar um **passe de 7 dias grátis**, e você ainda pode trazer um amigo! Vamos agendar sua primeira visita para hoje ou amanhã?"""
        elif script_key == 'Reagendamento_Objeção_Pessoal':
            objetivo_limpo = self.contexto_lead.get('objetivo_pessoal_completo', 'ser mais saudável').strip().rstrip('.')
            script = f"Entendi sua preocupação, [Nome]. Mas você mencionou que o objetivo é **{objetivo_limpo}**. Se você adiar, vai ficar mais difícil atingir essa meta. Vamos agendar sua visita para hoje ou amanhã para garantir que você não perca o foco?"
        elif script_key == 'Encerramento_Suave':
            script = "Que alegria saber que quer começar essa jornada com a gente, [Nome]! Vamos garantir que você alcance seus objetivos. Que tal deixar agendado sua visita para mês que vem? Prefere na primeira semana do mês ou segunda?"
        elif script_key == 'Objecao_Preco_Digital_Persistente':
            script = "Certo, [Nome]! Eu entendo a preocupação! Mas olha só: nossos planos começam a partir de R$ 49,90. Para que você tenha certeza de que é o investimento certo, posso te liberar 7 dias de acesso total ao ambiente e professores, sem compromisso. Topa vir conhecer e testar nosso trabalho?"
        else:
            script = SCRIPTS.get(script_key)

        nome_substituto = self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'
        script = script.replace('[Nome]', nome_substituto)
        return script

    def _get_script(self, intent_name):
        script = self._format_script(intent_name)

        # Não adiciona humanização se for uma transferência imediata ou se for o pedido inicial de nome
        if intent_name in ['Agendamento_Avaliacao', 'Interesse_Suplementos', 'Transferencia_Consultor', 'Interesse_Inicial']:
            return script

        return self.add_human_touch(script)


    def add_human_touch(self, response: str) -> str:
        if random.random() < 0.25:
            toque = random.choice(FRASES_HUMANIZACAO)
            return f"{response}\n\n[Humanização: {toque}]"
        return response

    def check_lead_history(self, telefone: str) -> Optional[str]:
        if telefone in CRM_INATIVOS and CRM_INATIVOS[telefone]['status'] == 'INATIVO':
            self.nome_lead = CRM_INATIVOS[telefone]['nome']
            return 'Inativo_Identificado'
        return None

    def _simular_nlp(self, message: str) -> str:
        message_lower = message.lower()

        # --- 1. PRIORIDADE DE FECHAMENTO CONTEXTUAL (V41) ---
        if self.contexto_lead['estado_atual'] in ['AGUARDANDO_AGENDAMENTO', 'AGUARDANDO_RESPOSTA_OBJECAO']:
             if any(keyword in message_lower for keyword in ['hoje', 'amanhã', 'tarde', 'manhã', 'agora', 'sim', 'quero', 'pode ser', 'consigo ir']):
                 return 'Agendamento_Sucesso'
        # --------------------------------------------------------

        # --- 2. ISOLAMENTO DE CAPTURA DE NOME (V39) ---
        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             # Se for uma Resposta_Nome ou se a mensagem for curta (para pegar nome simples), prioriza a captura.
             if 'nome' in message_lower or len(message.split()) <= 4:
                 return 'Resposta_Nome'
             # Se não for Resposta_Nome, mas for uma Intenção Imediata (ex: "quero avaliação"), deve ser tratada como Intenção Imediata
             # MAS NO ESTADO AGUARDANDO_NOME, QUALQUER COISA QUE NÃO FOR NOME, DEVE REPETIR O PEDIDO.
             return 'Intencao_Nao_Identificada'
        # ---------------------------------------------

        # --- 3. INTENÇÕES DE ALTA PRIORIDADE (Transversais) ---
        # Estas só são verificadas se o estado NÃO for AGUARDANDO_NOME
        if any(keyword in message_lower for keyword in ['avaliação física', 'agendar avaliação', 'como funciona avaliação']): return 'Agendamento_Avaliacao'
        if any(keyword in message_lower for keyword in ['suplementos', 'whey', 'creatina', 'comprar whey', 'vendem suplementos']): return 'Interesse_Suplementos'
        if any(keyword in message_lower for keyword in ['tchau', 'agradeço', 'não vou seguir', 'parar por aqui', 'vou sair', 'não quero de forma alguma']): return 'Recusa_Total'
        if ('preço' in message_lower or 'valor' in message_lower or 'caro' in message_lower or 'pesado' in message_lower): return 'Objecao_Preco_Digital'
        if ('tempo' in message_lower or 'corrido' in message_lower or 'sem tempo' in message_lower): return 'Objecao_Sem_Tempo'

        # --- 4. INTENÇÕES DE FLUXO SEQUENCIAL ---

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            termos_objetivo = ['músculos', 'musculos', 'musculo', 'emagrecer', 'saúde', 'condicionamento', 'foco', 'massa']
            if any(termo in message_lower for termo in termos_objetivo):
                self.contexto_lead['objetivo_pessoal_completo'] = message
                return 'Resposta_Objetivo'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            if any(keyword in message_lower for keyword in ['ex-aluno', 'já fui', 'treinei antes', 'antigo aluno']):
                self.contexto_lead['historico_treino'] = 'EX_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            if any(keyword in message_lower for keyword in ['nunca', 'primeira vez', 'começando agora']):
                self.contexto_lead['historico_treino'] = 'NOVO_ALUNO'
                return 'Resposta_Historico_Treino_Especifica'
            return 'Resposta_Historico_Treino_Geral'

        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
             return 'Resposta_Conhece_Academia'

        if self.contexto_lead['estado_atual'] == 'INICIO': return 'Interesse_Inicial'

        return 'Intencao_Nao_Identificada'

    def _handle_preco_objection(self):

        if self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self._get_script('Interesse_Inicial')

        self.contexto_lead['objecao_preco_contagem'] += 1
        contagem = self.contexto_lead['objecao_preco_contagem']

        if contagem == 1:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self._get_script('Reagendamento_Objeção_Pessoal')

        elif contagem == 2:
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_RESPOSTA_OBJECAO'
            return self._get_script('Objecao_Preco_Digital_Persistente')

        else: # contagem >= 3
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script('Transferencia_Consultor')


    def generate_response(self, client_message: str) -> str:

        if self.contexto_lead['estado_atual'] in ['TRANSFERIDO', 'FIM']:
             return self._get_script('Transferencia_Consultor')

        intent = self._simular_nlp(client_message)

        # 1. TRATAMENTO DE INTERESSE INICIAL (Máxima Prioridade)
        if self.contexto_lead['estado_atual'] == 'INICIO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_NOME'
            return self._get_script('Interesse_Inicial')

        # 2. CAPTURA DE NOME (Prioridade Máxima no estado AGUARDANDO_NOME)
        if self.contexto_lead['estado_atual'] == 'AGUARDANDO_NOME':
             if intent == 'Resposta_Nome':
                 self.contexto_lead['objecao_preco_contagem'] = 0

                 # Extração de nome (V40): sempre pelo último token
                 tokens = client_message.split()
                 nome_candidato = tokens[-1].replace('.', '').replace(',', '')

                 if len(nome_candidato) > 2 and nome_candidato.lower() not in ['sim', 'não', 'eu', 'ok', 'aqui']:
                     self.nome_lead = nome_candidato

                 self.contexto_lead['estado_atual'] = 'AGUARDANDO_OBJETIVO'
                 return self._get_script('Resposta_Objetivo')

             # Se a intenção não for Resposta_Nome (e não for 'INICIO'), repete o pedido.
             return self._get_script('Interesse_Inicial')

        # 3. FLUXO DE AGENDAMENTO BEM SUCEDIDO
        if intent == 'Agendamento_Sucesso':
            self.contexto_lead['estado_atual'] = 'FIM'
            return f"Perfeito, {self.nome_lead if self.nome_lead else 'futuro(a) aluno(a)'}! A consultora irá confirmar o melhor horário para você. Nos vemos na academia! 🥳"

        # 4. TRATAMENTO DE INTENÇÕES IMEDIATAS (Avaliação e Suplementos) - Apenas após a captura de nome
        if intent in ['Agendamento_Avaliacao', 'Interesse_Suplementos']:
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script(intent)

        # 5. TRATAMENTO DE OBJEÇÕES DE VENDA
        if intent == 'Objecao_Preco_Digital':
            return self._handle_preco_objection()

        if intent == 'Objecao_Sem_Tempo':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script(intent)

        # 6. FLUXO DE ENCERRAMENTO (Recusa total)
        if intent == 'Recusa_Total':
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script('Transferencia_Consultor')

        # 7. FLUXO SEQUENCIAL PRINCIPAL (Objetivo, Histórico, Visita)

        elif intent == 'Resposta_Objetivo' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_OBJETIVO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_HISTORICO'
            return self._get_script('Resposta_Historico_Treino')

        elif intent in ['Resposta_Historico_Treino_Especifica', 'Resposta_Historico_Treino_Geral'] and self.contexto_lead['estado_atual'] == 'AGUARDANDO_HISTORICO':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_VISITA'
            return self._get_script('Resposta_Conhece_Academia')

        elif intent == 'Resposta_Conhece_Academia' and self.contexto_lead['estado_atual'] == 'AGUARDANDO_VISITA':
            self.contexto_lead['estado_atual'] = 'AGUARDANDO_AGENDAMENTO'

            if self.contexto_lead.get('historico_treino') == 'EX_ALUNO':
                 return self._get_script('Offer_Ex_Aluno')
            else:
                 return self._get_script('Offer_Novo_Aluno')

        # 8. FALLBACK UNIVERSAL
        else:
            self.contexto_lead['estado_atual'] = 'TRANSFERIDO'
            return self._get_script('Transferencia_Consultor')


# --- FUNÇÕES DE TESTE ---

def testar_fluxo(agente_ia, nome_do_teste: str, mensagens: Dict[str, str]):
    """Função para simular um fluxo de conversa completo."""
    print("=====================================================")
    print(f"✅ INICIANDO TESTE: {nome_do_teste}")
    print("=====================================================")

    agente_ia.contexto_lead = {'estado_atual': 'INICIO', 'telefone': None, 'inativo_tratado': False, 'objecao_preco_contagem': 0, 'historico_treino': None, 'objetivo_pessoal_completo': 'ser mais saudável'}
    agente_ia.nome_lead = ''

    for idx, (cliente_msg, intent_esperada) in enumerate(mensagens.items()):
        agente_response_full = agente_ia.generate_response(cliente_msg)
        agente_response = agente_response_full.splitlines()[0] if agente_response_full else ''

        print(f"\n[{idx+1}] CLIENTE: {cliente_msg}")
        print(f"    AGENTE ({agente_ia.contexto_lead['estado_atual']}): {agente_response}...")

        if intent_esperada in agente_response_full:
             print(f"    [SUCESSO] Contém: '{intent_esperada.splitlines()[0]}'")
        else:
             print(f"    [FALHA] Resposta inesperada. Esperado conter: '{intent_esperada.splitlines()[0]}'")

# --- FLUXO DE TESTE DE BORDA ---

fluxo_borda_prioridade = {
    "Olá, quero saber sobre as aulas.": "Boa tarde!", # 1. INICIO -> AGUARDANDO_NOME
    "Quero agendar uma avaliação física.": "Boa tarde!", # 2. AGUARDANDO_NOME (Deveria repetir o pedido)
    "Ah, meu nome é Pedro.": "Pra eu te orientar direitinho, qual é o seu objetivo hoje?", # 3. AGUARDANDO_OBJETIVO (Captura nome e avança)
    "Queria só a avaliação física.": "Claro! Para a avaliação física, vou te transferir agora para nossa equipe técnica. Ela já verifica sua disponibilidade e agenda o melhor horário. Posso transferir em 1 minuto, Pedro?", # 4. TRANSFERIDO (Intenção Imediata só agora é aceita)
}


# --- EXECUÇÃO DO NOVO TESTE ---

print("\n--- TESTE DE COBERTURA FINAL (V42) ---")
agente_final = SalesAgentConsultivo()
testar_fluxo(agente_final, "Fluxo 5: Prioridade Máxima na Captura de Nome", fluxo_borda_prioridade)


--- TESTE DE COBERTURA FINAL (V42) ---
Agente Consultivo ACADE FINAL (V42 - Teste de Borda de Prioridade) inicializado.
✅ INICIANDO TESTE: Fluxo 5: Prioridade Máxima na Captura de Nome

[1] CLIENTE: Olá, quero saber sobre as aulas.
    AGENTE (AGUARDANDO_NOME): Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?...
    [SUCESSO] Contém: 'Boa tarde!'

[2] CLIENTE: Quero agendar uma avaliação física.
    AGENTE (AGUARDANDO_NOME): Boa tarde! 😊 Que bom seu contato. Antes de te passar qualquer informação, me diz uma coisa: qual é o seu nome?...
    [SUCESSO] Contém: 'Boa tarde!'

[3] CLIENTE: Ah, meu nome é Pedro.
    AGENTE (AGUARDANDO_OBJETIVO): Pra eu te orientar direitinho, qual é o seu objetivo hoje? • Emagrecer • Ganhar massa • Saúde • Condicionamento • Ou só voltar ao foco?...
    [SUCESSO] Contém: 'Pra eu te orientar direitinho, qual é o seu objetivo hoje?'

[4] CLIENTE: Queria só a avaliação física.
    AGENTE (TRANSFERIDO